# Understanding the `compileconfig` Decorator

The `compileconfig` decorator is a powerful tool for compiling and managing AI Engine kernels. It provides a flexible way to control the compilation process, including specifying source files, compilation flags, and even using pre-compiled object files.

## Basic Usage

The `compileconfig` decorator returns a `Compilable` object, which encapsulates the function and its compilation configuration. The `jit` decorator can then be used to execute the compiled kernel. The `jit` decorator returns a `Callable` object.

In [ ]:
import aie.iron as iron
import numpy as np


@iron.compileconfig()
def vector_add(input0, input1, output):
    # ... kernel implementation ...
    pass


print(f"'vector_add' is of type: {type(vector_add)}")


@iron.jit
def jitted_vector_add(input0, input1, output):
    return vector_add(input0, input1, output)


print(f"'jitted_vector_add' is of type: {type(jitted_vector_add)}")

## Accessing the `Compilable` Object

You can access the `Compilable` object used internally by the `jit` decorator through the `compilable` attribute of the decorated function.

In [ ]:
compilable_obj = jitted_vector_add.compilable
print(compilable_obj.to_json())

## The `Callable` Class

The `jit` decorator returns a `Callable` object. This object is callable, and when called, it will perform the JIT compilation and execution of the kernel.

In [ ]:
jitted_vector_add = iron.jit(vector_add)
assert isinstance(jitted_vector_add, iron.Callable)

## Stacking Decorators

You can stack the `jit` and `compileconfig` decorators to create a JIT-compiled function with a specific compilation configuration.

In [ ]:
@iron.jit
@iron.compileconfig(metaargs={"my_var": 42})
def my_jitted_kernel(input0, output):
    # ... kernel implementation ...
    pass

## Using Pre-compiled Artifacts with `jit`

You can also use a `PreCompiled` object with the `jit` decorator to create a JIT-compiled function that uses pre-compiled artifacts.

In [ ]:
precompiled = iron.PreCompiled(xclbin_path="path/to/my_kernel.xclbin", insts_path="path/to/my_kernel.insts")

@iron.jit
def my_precompiled_kernel(input0, output):
    return precompiled(input0, output)

## Compiling with External Source Files

You can include external C++ source files in the compilation by using the `source_files` argument. You can also provide compilation flags and include paths.

In [ ]:
@iron.compileconfig(
    source_files=["my_kernel.cpp"],
    compile_flags=["-O3"],
    include_paths=["/path/to/includes"],
)
def my_kernel_with_extern(input0, output):
    # ... kernel implementation ...
    pass

## Using Pre-compiled Object Files

If you have pre-compiled object files, you can use them directly by passing them to the `object_files` argument. This will skip the C++ compilation step.

In [ ]:
@iron.compileconfig(object_files=["my_kernel.o"])
def my_kernel_with_object_files(input0, output):
    # ... kernel implementation ...
    pass

## Compile-Time Arguments

Compile-time arguments allow you to control the compilation process using Python variables. You can define compile-time arguments by passing a dictionary to the `metaargs` argument. These variables are then available in the kernel implementation using the `get_compile_arg` function.

In [ ]:
@iron.compileconfig(metaargs={"my_var": 42})
def my_compile_arg_kernel(input0, output):
    my_var = iron.get_compile_arg("my_var")
    # ... use my_var in the kernel implementation ...
    pass

## JSON Serialization

A `Compilable` object can be serialized to and from JSON, which is useful for storing and sharing compilation configurations.

In [ ]:
compilable = iron.compileconfig(my_kernel_with_extern)
json_str = compilable.to_json()
print(json_str)

new_compilable = iron.Compilable.from_json(json_str, my_kernel_with_extern)

### JSON Output

## Getting Artifacts

You can get the paths to the compilation artifacts using the `get_artifacts` method of the `Compilable` object.

In [ ]:
jitted_vector_add(
    np.ones(16, dtype=np.int32),
    np.ones(16, dtype=np.int32),
    np.zeros(16, dtype=np.int32),
)
xclbin_path, insts_path = jitted_vector_add.compilable.get_artifacts()
print(f"xclbin path: {xclbin_path}")
print(f"insts path: {insts_path}")